In [19]:
import warnings
import pandas as pd
import numpy as np
import os
import sys # error msg
import operator # sorting
from math import *

from read_trace import *
from avgblkmodel import *

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

# gpu info

In [20]:
gtx950 = DeviceInfo()
gtx950.sm_num = 6
gtx950.sharedmem_per_sm = 49152
gtx950.reg_per_sm = 65536
gtx950.maxthreads_per_sm = 2048

# single stream info

In [21]:
data_size = 23000
trace_file = './1cke/trace_' + str(data_size) + '.csv'
df_trace = trace2dataframe(trace_file) # read the trace to the dataframe

In [22]:
df_trace

,Start,Duration,Grid X,Grid Y,Grid Z,Block X,Block Y,Block Z,Registers Per Thread,Static SMem,Dynamic SMem,Size,Throughput,Device,Context,Stream,Name
0,ms,us,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,B,KB,GB/s,NaN,NaN,NaN,NaN
1,526.961828,16.672000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.843750,5.139256,GeForce GTX 950 (0),1.0,13.0,[CUDA memcpy HtoD]
2,526.979716,16.224000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.843750,5.281168,GeForce GTX 950 (0),1.0,13.0,[CUDA memcpy HtoD]
3,527.157829,61.056000,90.0,1.0,1.0,256.0,1.0,1.0,28.0,0,0,NaN,NaN,GeForce GTX 950 (0),1.0,13.0,"kernel_vectorAdd(float const *, float const *,..."
4,527.221349,15.904000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.843750,5.387429,GeForce GTX 950 (0),1.0,13.0,[CUDA memcpy DtoH]


In [23]:
ApiList = ["h2d", "h2d_ovhd", "kern_laun_ovhd", "kern", "kern_ovhd", "d2h_laun_ovhd", "d2h", "d2h_ovhd"]

class Node:
    """
    Record the api type and timing for each trace.
    """
    def __init__(self, api_type, start_ms, end_ms):
        api_type_str = str(api_type).lower()
        if api_type_str in ApiList:
            self.api_type = api_type_str
        else:
            sys.stderr.write("Error! Unknown api type.\n{}\n".format(ApiList))
            
        self.start = float(start_ms) # make sure it is float
        self.end = float(end_ms)
        self.next = None
        self.prev = None

In [24]:
class StreamTime():
    """
    A linked list to connect all the cuda api calls.
    """
    def __init__ (self):
        self.head = None
        
    def add(self, api_type, start_ms, end_ms):
        node = Node(api_type, start_ms, end_ms)
        if self.head == None:
            self.head = node
        else:
            node.prev = self.head
            node.prev.next = node
            self.head = node  

In [25]:
stream_timing = StreamTime()

stream_timing.add('h2d', 0,2)
stream_timing.add('h2d_ovhd', 2,2.1)
stream_timing.add('h2d', 2.1,3)


In [6]:
start_coef, duration_coef = time_coef_ms(df_trace) # convert time to ms
print('{} {}'.format(start_coef, duration_coef))
ssm_coef, dsm_coef = sm_coef_bytes(df_trace) # convert shared mem to bytes
print('{} {}'.format(ssm_coef, dsm_coef))

1.0 0.001
1.0 1.0


In [7]:
stream_id_list = df_trace['Stream'].unique()
stream_id_list = stream_id_list[~np.isnan(stream_id_list)] # remove nan
num_streams = len(stream_id_list)
print('number of streams : {}'.format(num_streams))

streamList = [StreamAPI() for i in range(num_streams)]

number of streams : 1


In [8]:
len(streamList)

1

In [9]:
# def get_stream_info(df_trace):
#     """
#     read dataframe into stream list which contains the h2d/d2h/kernel star and end time in ms.
#     """

#     # read row by row
#     for rowID in xrange(1, df_trace.shape[0]):
#         #  extract info from the current row
#         stream_id, api_type, start_time_ms, end_time_ms, kerninfo = read_row(df_trace.iloc[[rowID]], start_coef, duration_coef, ssm_coef, dsm_coef)

#         # find out index of the stream
#         sid, = np.where(stream_id_list==stream_id)

#         # add the start/end time for different api calls
#         if api_type == 'h2d':
#             streamList[sid].h2d.append(transfer(start_time_ms, end_time_ms))
#         elif api_type == 'd2h':
#             streamList[sid].d2h.append(transfer(start_time_ms, end_time_ms))
#         elif api_type == 'kernel':
#             streamList[sid].kernel.append(transfer(start_time_ms, end_time_ms))
#             streamList[sid].kernel_info.append(kerninfo) # add the kernel info
#         else:
#             print "Unknown. Error."

#     return streamList